In [ ]:
# !pip install -U pystad

In [2]:
import json
import stad as sd
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from pyArango.connection import *
import networkx as nx

In [5]:
# format of credentials.json file that should be in the directory in which you are running this script. (To connect to personal ArangoDB environemnt)
#     { "url": "http://localhost:8529",
#       "databaseName": "",
#       "auth": { "username": "", "password": "
#     }

credentials = json.load(open('credentials.json'))
print(credentials)

{'url': 'http://localhost:8529', 'databaseName': '_system', 'auth': {'username': 'root', 'password': ''}}


In [6]:
#To check whether connection was succesfull
conn = Connection(username=credentials["auth"]["username"], password=credentials["auth"]["password"])
db = conn[credentials["databaseName"]]
db

ArangoDB database: _system

In [8]:
# collection 'murih_samples' looks like this:
# [{
#     "_key": "IAM01_1",
#     "_id": "murih_samples/IAM01_1",
#     "_rev": "_dimFSyO---",
#     "individual": "IAM01",
#     "age": 58,
#     "gender": "M",
#     "X__Saturatie": -1.090955188,
#     "Albumine": -0.688442937,
#     "Albumine____": 0.706384552,
#     "Aldosteron": 1.598158214,
#     "Alfa_1_globulinen": -1.139896645,
#     "Alfa_2_globulinen": 0.34075838,
#     ...

aql = "FOR x IN bioMOBS_test_samples RETURN x"
queryResult = db.AQLQuery(aql, rawResults=True, batchSize=100)
data = pd.DataFrame(queryResult)
data

_key                          _id         _rev  Apolipoprotein.B  \
0    ID.11   bioMOBS_test_samples/ID.11  _f_4HKI6---                 8   
1    ID.12   bioMOBS_test_samples/ID.12  _f_4HKI6--_                 6   
2    ID.13   bioMOBS_test_samples/ID.13  _f_4HKI6--A                28   
3    ID.14   bioMOBS_test_samples/ID.14  _f_4HKI6--B                25   
4    ID.15   bioMOBS_test_samples/ID.15  _f_4HKI6--C                 9   
5    ID.16   bioMOBS_test_samples/ID.16  _f_4HKI6--D                 4   
6    ID.17   bioMOBS_test_samples/ID.17  _f_4HKI6--E                19   
7    ID.18   bioMOBS_test_samples/ID.18  _f_4HKI6--F                17   
8    ID.19   bioMOBS_test_samples/ID.19  _f_4HKI6--G                 9   
9   ID.110  bioMOBS_test_samples/ID.110  _f_4HKI6--H                15   
10  ID.111  bioMOBS_test_samples/ID.111  _f_4HKI6--I                 8   
11  ID.112  bioMOBS_test_samples/ID.112  _f_4HKI6--J                27   
12  ID.113  bioMOBS_test_samples/ID.113  _f_4HKI6--K                17   
13  ID.114  bioMOBS_test_samples/ID.114  _f_4HKI6--L                24   
14  ID.115  bioMOBS_test_samples/ID.115  _f_4HKI6--M                29   
15  ID.116  bioMOBS_test_samples/ID.116  _f_4HKI6--N                10   
16  ID.117  bioMOBS_test_samples/ID.117  _f_4HKI6--O                15   
17  ID.118  bioMOBS_test_samples/ID.118  _f_4HKI6--P                 2   
18  ID.119  bioMOBS_test_samples/ID.119  _f_4HKJ----                 4   
19  ID.120  bioMOBS_test_samples/ID.120  _f_4HKJ---_                10   
20  ID.121  bioMOBS_test_samples/ID.121  _f_4HKJ---A                10   
21  ID.122  bioMOBS_test_samples/ID.122  _f_4HKJ---B                21   
22  ID.123  bioMOBS_test_samples/ID.123  _f_4HKJ---C                11   
23  ID.124  bioMOBS_test_samples/ID.124  _f_4HKJ---D                 6   
24  ID.125  bioMOBS_test_samples/ID.125  _f_4HKJ---E                14   
25  ID.126  bioMOBS_test_samples/ID.126  _f_4HKJ---F                17   
26  ID.127  bioMOBS_test_samples/ID.127  _f_4HKJ---G                 6   
27  ID.128  bioMOBS_test_samples/ID.128  _f_4HKJ---H                11   
28  ID.129  bioMOBS_test_samples/ID.129  _f_4HKJ---I                 6   
29  ID.130  bioMOBS_test_samples/ID.130  _f_4HKJ---J                22   
30  ID.131  bioMOBS_test_samples/ID.131  _f_4HKJ---K                23   
31  ID.132  bioMOBS_test_samples/ID.132  _f_4HKJ---L                 6   
32  ID.133  bioMOBS_test_samples/ID.133  _f_4HKJ---M                 9   
33  ID.134  bioMOBS_test_samples/ID.134  _f_4HKJ---N                29   
34  ID.135  bioMOBS_test_samples/ID.135  _f_4HKJ---O                15   
35  ID.136  bioMOBS_test_samples/ID.136  _f_4HKJ---P                23   
36  ID.137  bioMOBS_test_samples/ID.137  _f_4HKJ---Q                15   
37  ID.138  bioMOBS_test_samples/ID.138  _f_4HKJ---R                 9   
38  ID.139  bioMOBS_test_samples/ID.139  _f_4HKJ---S                 9   
39  ID.140  bioMOBS_test_samples/ID.140  _f_4HKJ---T                28   
40  ID.141  bioMOBS_test_samples/ID.141  _f_4HKJ---U                25   
41  ID.142  bioMOBS_test_samples/ID.142  _f_4HKJ---V                13   
42  ID.143  bioMOBS_test_samples/ID.143  _f_4HKJ---W                20   
43  ID.144  bioMOBS_test_samples/ID.144  _f_4HKJ---X                25   
44  ID.145  bioMOBS_test_samples/ID.145  _f_4HKJ---Y                29   
45  ID.146  bioMOBS_test_samples/ID.146  _f_4HKJ---Z                 2   
46  ID.147  bioMOBS_test_samples/ID.147  _f_4HKJC---                13   
47  ID.148  bioMOBS_test_samples/ID.148  _f_4HKJC--_                 2   
48  ID.149  bioMOBS_test_samples/ID.149  _f_4HKJC--A                13   
49  ID.150  bioMOBS_test_samples/ID.150  _f_4HKJC--B                25   

    Aspartate.aminotransferase  Direct.bilirubin  Urea  Calcium  Cholesterol  \
0                            3                19    11        3           11   
1                           24                26    20        6        

In [ ]:
df = pd.read_csv('cosine_biomobs.csv', sep=',', encoding='latin-1')
df

Unnamed: 0      ID 1      ID 2      ID 3      ID 4      ID 5      ID 6  \
0        ID 1  1.000000  0.829136  0.825167  0.756040  0.751056  0.764500   
1        ID 2  0.829136  1.000000  0.742665  0.767252  0.772240  0.788516   
2        ID 3  0.825167  0.742665  1.000000  0.830013  0.679622  0.755666   
3        ID 4  0.756040  0.767252  0.830013  1.000000  0.800075  0.762017   
4        ID 5  0.751056  0.772240  0.679622  0.800075  1.000000  0.726107   
5        ID 6  0.764500  0.788516  0.755666  0.762017  0.726107  1.000000   
6        ID 7  0.784106  0.789410  0.676661  0.762410  0.840903  0.656295   
7        ID 8  0.692114  0.692823  0.799388  0.753622  0.678031  0.748653   
8        ID 9  0.743927  0.766610  0.714487  0.753470  0.777824  0.736390   
9       ID 10  0.673107  0.734362  0.678492  0.789427  0.791319  0.734306   
10      ID 11  0.718781  0.732405  0.735310  0.804824  0.699603  0.731725   
11      ID 12  0.836944  0.754580  0.795761  0.743491  0.733200  0.729983   
12      ID 13  0.694493  0.817212  0.658705  0.729457  0.797094  0.701435   
13      ID 14  0.791799  0.813953  0.824450  0.790749  0.786692  0.835327   
14      ID 15  0.749731  0.757651  0.775093  0.838590  0.681286  0.736666   
15      ID 16  0.837183  0.823968  0.757599  0.767980  0.818256  0.642848   
16      ID 17  0.716727  0.779012  0.713284  0.771391  0.731842  0.679411   
17      ID 18  0.754851  0.829346  0.735364  0.778627  0.756004  0.774472   
18      ID 19  0.836558  0.928877  0.784412  0.797374  0.803964  0.851030   
19      ID 20  0.812749  0.769024  0.811739  0.811244  0.848051  0.764811   
20      ID 21  0.723668  0.818411  0.589584  0.733666  0.692659  0.741170   
21      ID 22  0.706030  0.749485  0.738031  0.769833  0.705363  0.771305   
22      ID 23  0.777074  0.845801  0.649805  0.741180  0.715126  0.787087   
23      ID 24  0.794152  0.743859  0.776426  0.772516  0.794994  0.635460   
24      ID 25  0.734950  0.826583  0.772180  0.785904  0.732140  0.824935   
25      ID 26  0.764583  0.783332  0.740335  0.779272  0.749302  0.710954   
26      ID 27  0.668353  0.816108  0.686064  0.766101  0.784135  0.771031   
27      ID 28  0.831609  0.910761  0.750377  0.835561  0.775704  0.865666   
28      ID 29  0.729049  0.857737  0.720132  0.790868  0.734728  0.747905   
29      ID 30  0.724861  0.822770  0.757111  0.795521  0.794194  0.728101   
30      ID 31  0.704634  0.716220  0.768710  0.738304  0.690735  0.776249   
31      ID 32  0.668099  0.831230  0.634730  0.776938  0.806964  0.763477   
32      ID 33  0.792965  0.812116  0.790837  0.837677  0.681467  0.776636   
33      ID 34  0.654691  0.761410  0.805633  0.790634  0.709057  0.678381   
34      ID 35  0.722226  0.815121  0.678105  0.706828  0.758819  0.689475   
35      ID 36  0.713008  0.785252  0.806294  0.787470  0.672301  0.690593   
36      ID 37  0.694706  0.803938  0.698186  0.833242  0.723487  0.653404   
37      ID 38  0.793152  0.887816  0.698167  0.742613  0.842524  0.830524   
38      ID 39  0.745236  0.849578  0.827426  0.830667  0.771930  0.821978   
39      ID 40  0.777898  0.894881  0.827063  0.790538  0.794749  0.778543   
40      ID 41  0.778344  0.731846  0.787083  0.743893  0.714739  0.745534   
41      ID 42  0.685161  0.775249  0.758887  0.646795  0.698999  0.754513   
42      ID 43  0.764128  0.754116  0.805742  0.767745  0.768282  0.783606   
43      ID 44  0.747022  0.790754  0.797460  0.823586  0.734471  0.679670   
44      ID 45  0.674105  0.843947  0.679913  0.782237  0.761675  0.743764   
45      ID 46  0.634287  0.818965  0.757535  0.748793  0.690615  0.802051   
46      ID 47  0.737413  0.869455  0.753567  0.797576  0.708859  0.740876   
47      ID 48  0.850618  0.855680  0.815880  0.761340  0.802584  0.828865   
48      ID 49  0.731886  0.739281  0.674749  0.758324  0.715868  0.565522   
49      ID 50  0.773664  0.765269  0.777153  0.878579  0.890864  0.726386   

        ID 7      ID 8      ID 9  ...     ID 41     ID 42     ID 43     ID 

In [ ]:
df = df.drop(columns="Unnamed: 0")
df.index = df.columns
similarities = df.to_numpy()
similarities

In [ ]:
euc_observations_square = 1- similarities
euc_observations = squareform(euc_observations_square)
euc_observations

In [ ]:
#this is the acutal stad computation 
network_observations, sweep_observations = sd.stad(euc_observations)

In [ ]:
adjacency_matrix = network_observations.toarray()
adjacency_matrix

In [ ]:
#Can take some time to export as well
import numpy as numpy
numpy.savetxt("adjacency_testbiomobs.csv", adjacency_matrix, delimiter=",")

In [ ]:
#empty Edge type biomobs_Test_STAD file needs to be created in ArangoDB environment
edgesCollection = db["biomobs_test_STAD"]
edgesCollection.truncate()
docs = []
for i in range(len(np.nonzero(adjacency_matrix)[0])):
    x = np.nonzero(adjacency_matrix)[0][i]
    y = np.nonzero(adjacency_matrix)[1][i]
    x_key = data.iloc[x]["_key"]
    y_key = data.iloc[y]["_key"]

    doc = edgesCollection.createDocument()
    doc['_key'] = x_key + '_' + y_key
    doc['_from'] = "bioMOBS_test_samples/" + x_key
    doc['_to'] = "bioMOBS_test_samples/" + y_key
    doc["weight"] = adjacency_matrix[x][y]
    docs.append(doc)

In [ ]:
#The crucial part of code do not forget this cell to put it actually in arangODB
edgesCollection.bulkSave(docs) 